In [1]:
%load_ext memory_profiler
from textar import TextClassifier
import xml.etree.ElementTree as ET
from lxml import etree
import numpy as np
import re
import os

In [28]:
# Helper funcs

def parse_blog(tree, min_words=100):
    dates = []
    posts = []
    for elem in tree:
        post = None
        if elem.tag == 'date':
            date = elem.text
        elif elem.tag == 'post':
            post = elem.text
        if post is not None: 
            words = re.findall('\w+\W',post)
            if len(words) > min_words and np.mean(map(len,words))>2:
                dates.append(date)
                posts.append(post)
    return dates, posts

In [29]:
# Configs
DATA_FOLDER = os.path.join('.','data','performance_data','blogs')
MAX_FILES = 10000

In [30]:
magic = '''<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
            "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd" [
            <!ENTITY nbsp ' '>
            ]>'''

parser = etree.XMLParser(recover=True)

all_dates = []
all_posts = []
all_genders = []
all_ages = []
all_categories = []

for file_name in os.listdir(DATA_FOLDER)[:MAX_FILES]:
    id_f, gender, age, category, zodiac, ext = file_name.split('.')
    with open(os.path.join(DATA_FOLDER, file_name), 'r') as f:
        try:
            tree = ET.fromstring(magic + f.read(), parser=parser)
            dates, posts = parse_blog(tree)
            all_posts += posts
            all_dates += dates
            all_genders += [gender] * len(dates)
            all_ages += [age] * len(dates)
            all_categories += [category] * len(dates)
        except Exception as e:
            pass
            #print("Error en {:s}".format(file_name))
all_ids = map(str, range(len(all_posts)))

In [31]:
%%timeit
# Tiempo de la creacion del objeto
tc = TextClassifier(all_posts, all_ids)

In [36]:
%%timeit
# Tiempo de la busqueda
tc.get_similar(all_ids[1],max_similars=3, term_diff_max_rank=50)

1 loop, best of 3: 2.36 s per loop


In [39]:
%%timeit
# Tiempo de creacion del clasificador
tc.make_classifier("topic",all_ids, all_categories)

1 loop, best of 3: 17.4 s per loop


In [50]:
%%timeit
tc.classify("topic", all_ids[1])

10 loops, best of 3: 31.4 ms per loop


In [18]:
row.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

array(['0', '1', '10', ..., '997', '998', '999'], 
      dtype='|S4')